In [ ]:
import lightgbm as lgb
from ticket_upgrade_prediction.pipeline import Pipeline

In [ ]:
d = Pipeline(model_type="predict_upgrade")
dataset = d.scale_final_dataset()

In [ ]:
model = lgb.LGBMClassifier()
model.fit(dataset.X_train, dataset.y_train)
print(model)
expected_y  = dataset.y_test
predicted_y = model.predict(dataset.X_test)

In [ ]:
from sklearn.metrics import classification_report
classification_report(expected_y, predicted_y)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(expected_y, predicted_y)